In [0]:
df_icd_codes=spark.read.parquet('/mnt/bronze/icd_codes/*.parquet')

df_icd_codes.createOrReplaceTempView('icd_codes')

In [0]:
%sql
CREATE TABLE IF NOT EXISTS healthcarerevenuecyclemanagement_databricks.silver.icd_codes(
  icd_code STRING,
  icd_code_type STRING,
  code_description STRING,
  inserted_date DATE,
  updated_date DATE,
  is_current_flag BOOLEAN
)

In [0]:
%sql
MERGE INTO healthcarerevenuecyclemanagement_databricks.silver.icd_codes AS target
USING icd_codes AS source 
ON target.icd_code = source.icd_code
WHEN MATCHED THEN 
    UPDATE 
    SET target.is_current_flag = False,
        target.updated_date = current_date()
WHEN NOT MATCHED
    THEN INSERT (
        icd_code,
        icd_code_type,
        code_description,
        inserted_date,
        updated_date,
        is_current_flag
    ) VALUES (
        source.icd_code,
        source.icd_code_type,
        source.code_description,
        source.inserted_date,
        source.updated_date,
        source.is_current_flag
    )